<a href="https://colab.research.google.com/github/AYCDJ/Cyber-Intelligence-assingment-s-/blob/main/CI_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

# Load the dataset and display basic information
file_path = "uci_malware_detection.csv"
df = pd.read_csv(file_path)
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Columns: 532 entries, Label to F_531
dtypes: int64(531), object(1)
memory usage: 1.5+ MB


(None,
            Label  F_1  F_2  F_3  F_4  F_5  F_6  F_7  F_8  F_9  ...  F_522  \
 0  non-malicious    1    0    1    0    1    0    1    0    1  ...      0   
 1  non-malicious    1    0    1    0    1    0    1    0    1  ...      0   
 2  non-malicious    1    0    1    0    1    0    1    0    1  ...      0   
 3  non-malicious    1    0    1    0    1    0    1    0    1  ...      0   
 4  non-malicious    1    0    1    0    1    0    1    0    1  ...      0   
 
    F_523  F_524  F_525  F_526  F_527  F_528  F_529  F_530  F_531  
 0      0      0      0      0      0      0      0      0      0  
 1      0      0      0      0      0      0      0      0      0  
 2      0      0      0      0      0      0      0      0      0  
 3      0      0      0      0      0      0      0      0      0  
 4      0      0      0      0      0      0      0      0      0  
 
 [5 rows x 532 columns])

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode the label column (malicious = 1, non-malicious = 0)
df['Label'] = LabelEncoder().fit_transform(df['Label'])

# Check the class distribution
class_counts = df['Label'].value_counts()

# Split features and labels into training and test sets (80% train, 20% test)
X = df.drop(columns=['Label'])
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Display class distribution and dataset shape
class_counts, X_train.shape, X_test.shape


(Label
 0    301
 1     72
 Name: count, dtype: int64,
 (298, 531),
 (75, 531))

In [10]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
resampled_class_counts = pd.Series(y_train_resampled).value_counts()
resampled_class_counts


,count
Label,
0,240
1,240


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Define models, decision tree, random forest, and svm
models = {
    "Decision Tree": DecisionTreeClassifier(class_weight="balanced", random_state=42),
    "Random Forest": RandomForestClassifier(class_weight="balanced", n_estimators=100, random_state=42),
    "SVM": SVC(class_weight="balanced", probability=True, random_state=42)
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store results
results = {}

for model_name, model in models.items():
    print(f"Training {model_name}...")

    # Cross-validation predictions and performance metrics
    y_pred = cross_val_predict(model, X_train_selected, y_train, cv=cv)

    accuracy = accuracy_score(y_train, y_pred)
    precision = precision_score(y_train, y_pred)
    recall = recall_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred)
    roc_auc = roc_auc_score(y_train, y_pred)

    results[model_name] = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1,
        "ROC-AUC": roc_auc
    }

# Convert results to a DataFrame for visualization
results_df = pd.DataFrame(results).T
print("\nModel Performance:\n", results_df)


Training Decision Tree...
Training Random Forest...
Training SVM...

Model Performance:
                Accuracy  Precision    Recall  F1-score   ROC-AUC
Decision Tree  0.996644        1.0  0.982759  0.991304  0.991379
Random Forest  0.993289        1.0  0.965517  0.982456  0.982759
SVM            0.993289        1.0  0.965517  0.982456  0.982759
